In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.4.2
2.8.2


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation


def sum_over(*args):
    return sum(args)

In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x00000171FB13D4C0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x00000171FB13D4C0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x00000171FB13D4C0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [8]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_0_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 2",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 0, "A.1_DikeIncrease 0": 5})
    ),
    Policy(
        "policy 3",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 0, "A.1_DikeIncrease 0": 10})
    ),
    Policy(
        "policy 4",
        **dict(get_0_dict(), **{"0_RfR 0": 1, "0_RfR 1": 0, "0_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 5",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 1, "0_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 6",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 1, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 7",
        **dict(get_0_dict(), **{"1_RfR 0": 1, "1_RfR 1": 0, "1_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 8",
        **dict(get_0_dict(), **{"1_RfR 0": 0, "1_RfR 1": 1, "1_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 9",
        **dict(get_0_dict(), **{"1_RfR 0": 0, "1_RfR 1": 0, "1_RfR 2": 1, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 10",
        **dict(get_0_dict(), **{"2_RfR 0": 1, "2_RfR 1": 0, "2_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 11",
        **dict(get_0_dict(), **{"2_RfR 0": 1, "2_RfR 1": 0, "2_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 12",
        **dict(get_0_dict(), **{"2_RfR 0": 1, "2_RfR 1": 0, "2_RfR 2": 0, "A.1_DikeIncrease 0": 0})
    ),
]

In [9]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle1 = True

if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    n_scenarios = 25
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, policies)

    # Save results in Pickle file
    with open('data/formulation_results.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)

In [10]:
import pandas as pd

a, b = results
df = pd.DataFrame(b)
df["policy"] = pd.DataFrame(a)["policy"]
df

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,policy
0,5.732935e+07,0.056965,4.356490e+07,0.057295,3.224118e+07,0.080806,2.908736e+07,0.018946,2.715114e+06,0.003386,0.0,0.0,policy 1
1,5.310821e+08,0.344665,5.858167e+06,0.005317,1.744272e+09,2.775271,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,policy 1
2,2.136219e+09,1.392643,1.669888e+08,0.141657,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,policy 1
3,0.000000e+00,0.000000,6.250319e+07,0.082046,1.197923e+09,2.817859,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,policy 1
4,0.000000e+00,0.000000,2.679906e+07,0.022682,5.172838e+08,0.785246,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,policy 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.000000e+00,0.000000,2.148880e+07,0.022244,4.020331e+07,0.076890,1.625602e+06,0.000968,2.870248e+08,0.275452,30700000.0,0.0,policy 12
296,2.347294e+09,1.529216,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,30700000.0,0.0,policy 12
297,2.126107e+09,1.542904,0.000000e+00,0.000000,0.000000e+00,0.000000,4.445023e+06,0.001737,0.000000e+00,0.000000,30700000.0,0.0,policy 12
298,2.211942e+09,1.517858,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,30700000.0,0.0,policy 12


In [11]:
def q05(x):
    return x.quantile(0.05)


def q95(x):
    return x.quantile(0.95)

In [12]:
group_df = df.groupby(by="policy")
agg_par = ["mean", "min", "max", q05, q95]
agg_dict = {
    "A.3 Total Costs": agg_par,
    "A.3_Expected Number of Deaths": agg_par,
    "RfR Total Costs": agg_par,
    "Expected Evacuation Costs": agg_par,
}
TCA_df = group_df.agg(agg_dict)
TCA_df

A.3 Total Costs                                        \
                     mean  min           max  q05           q95   
policy                                                            
policy 1     3.326075e+08  0.0  1.744272e+09  0.0  1.570691e+09   
policy 10    2.446031e+08  0.0  1.641221e+09  0.0  1.478844e+09   
policy 11    2.446031e+08  0.0  1.641221e+09  0.0  1.478844e+09   
policy 12    2.446031e+08  0.0  1.641221e+09  0.0  1.478844e+09   
policy 2     4.373169e+08  0.0  1.746347e+09  0.0  1.702134e+09   
policy 3     4.405365e+08  0.0  1.746347e+09  0.0  1.702134e+09   
policy 4     4.297020e+08  0.0  1.746347e+09  0.0  1.700258e+09   
policy 5     4.297020e+08  0.0  1.746347e+09  0.0  1.700258e+09   
policy 6     4.297020e+08  0.0  1.746347e+09  0.0  1.700258e+09   
policy 7     4.296421e+08  0.0  1.746347e+09  0.0  1.699323e+09   
policy 8     4.296421e+08  0.0  1.746347e+09  0.0  1.699323e+09   
policy 9     4.296421e+08  0.0  1.746347e+09  0.0  1.699323e+09   

          A.3_Expected Number of Deaths                                \
                                   mean  min       max  q05       q95   
policy                                                                  
policy 1                       0.616094  0.0  2.817859  0.0  2.772051   
policy 10                      0.442882  0.0  2.527617  0.0  2.497275   
policy 11                      0.442882  0.0  2.527617  0.0  2.497275   
policy 12                      0.442882  0.0  2.527617  0.0  2.497275   
policy 2                       0.805723  0.0  2.833872  0.0  2.810590   
policy 3                       0.810759  0.0  2.833872  0.0  2.810590   
policy 4                       0.792352  0.0  2.826466  0.0  2.810590   
policy 5                       0.792352  0.0  2.826466  0.0  2.810590   
policy 6                       0.792352  0.0  2.826466  0.0  2.810590   
policy 7                       0.792167  0.0  2.823023  0.0  2.810590   
policy 8                       0.792167  0.0  2.823023  0.0  2.810590   
policy 9                       0.792167  0.0  2.823023  0.0  2.810590   

          RfR Total Costs                                                      \
                     mean          min          max          q05          q95   
policy                                                                          
policy 1              0.0          0.0          0.0          0.0          0.0   
policy 10      30700000.0   30700000.0   30700000.0   30700000.0   30700000.0   
policy 11      30700000.0   30700000.0   30700000.0   30700000.0   30700000.0   
policy 12      30700000.0   30700000.0   30700000.0   30700000.0   30700000.0   
policy 2              0.0          0.0          0.0          0.0          0.0   
policy 3              0.0          0.0          0.0          0.0          0.0   
policy 4       84600000.0   84600000.0   84600000.0   84600000.0   84600000.0   
policy 5       84600000.0   84600000.0   84600000.0   84600000.0   84600000.0   
policy 6       84600000.0   84600000.0   84600000.0   84600000.0   84600000.0   
policy 7      217800000.0  217800000.0  217800000.0  217800000.0  217800000.0   
policy 8      217800000.0  217800000.0  217800000.0  217800000.0  217800000.0   
policy 9      217800000.0  217800000.0  217800000.0  217800000.0  217800000.0   

          Expected Evacuation Costs                      
                               mean  min  max  q05  q95  
policy                                                   
policy 1                        0.0  0.0  0.0  0.0  0.0  
policy 10                       0.0  0.0  0.0  0.0  0.0  
policy 11                       0.0  0.0  0.0  0.0  0.0  
policy 12                       0.0  0.0  0.0  0.0  0.0  
policy 2                        0.0  0.0  0.0  0.0  0.0  
policy 3                        0.0  0.0  0.0  0.0  0.0  
policy 4                        0.0  0.0  0.0  0.0  0.0  
policy 5                        0.0  0.0  0.0  0.0  0.0  
policy 6                        0.0  0.0  0.0  0.0  0.

In [13]:
TCA_df.to_csv("results/TCA3_results.csv")